<a href="https://colab.research.google.com/github/christopher-hsu/latent-map-planning/blob/master/model/jupyter_notebooks/InterpolateSiameseCVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Shared by all
import os, glob
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils # We should use this eventually.
from torch import nn, optim
from torch.nn import functional as F
import numpy as np


# For DataLoader
from PIL import Image
import numbers

import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')     

In [10]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Dataset Code: `CarlaDataset.py`

In [0]:
class CarlaDataset(Dataset):
    def __init__(self, data_dir, load_as_grayscale=False, transform=None):
        # xcxc I'm assuming that the images live in _out.
        self.data_dir = data_dir
        self.transform = transform
        self.load_as_grayscale = load_as_grayscale
        self.df = self._get_dataframe()
        
    def __len__(self):
        num_rows, _ = self.df.shape
        return num_rows
    
    def __getitem__(self, idx):
        '''
        Generate one sample of data.
        '''
        # We're gonna do some hardcore hard-coding here.
        # First, extract our control inputs
        row = self.df.iloc[idx]
        imgs_t0, imgs_t1 = self._get_image_tensor_for_row(row)
        control_inputs = np.array([row['ctr1'], row['ctr2']])
    
        return (imgs_t0, imgs_t1, control_inputs)
    
    def _get_image_tensor_for_row(self, row):
        '''
        Inputs:
            row_id: String that represents the input_num
        Outputs:
            A (2 x H x W x 4) 4D matrix of the two images.
        '''
        # The row_id should be the input_num. Should also be a string.
        img_t0_filenames = [row['img1_t0'], row['img2_t0']]
        img_t1_filenames = [row['img1_t1'], row['img2_t1']]
        img_t0_filenames.sort()
        img_t1_filenames.sort()
        t0_images = []
        for ele in img_t0_filenames:
            full_name = os.path.join(self.data_dir, '_out', ele)
            img_as_np_arr = self._load_image_and_maybe_apply_transform(full_name)
            t0_images.append(img_as_np_arr)
        t1_images = []
        for ele in img_t1_filenames:
            full_name = os.path.join(self.data_dir, '_out', ele)
            img_as_np_arr = self._load_image_and_maybe_apply_transform(full_name)
            t1_images.append(img_as_np_arr)
        
        t0_images = np.array(t0_images)
        t1_images = np.array(t1_images)
        return t0_images, t1_images

    def _load_image_and_maybe_apply_transform(self, image_loc):
        '''
        Inputs:
            image_loc: The location of the image we want to load
        Outputs:
            Either the grayscale image, a RGB image with the axes flopped, or 
            the RGB image with some series of transformations applied. 
            All are converted to numpy arrays before yeeting them out.
        '''
        # I've been writing too much haskell
        pil_img = Image.open(image_loc)
        if self.load_as_grayscale:
            pil_img = pil_img.convert('L')
        
        if self.transform:
            transform_result = self.transform(pil_img)
            return np.asarray(transform_result[:3, :, :])
        else:
            if self.load_as_grayscale:
                return np.array(pil_img)
            else:
                return self._rearrange_axes_image(np.array(pil_img))

    def _rearrange_axes_image(self, img):
        H,W,_ = img.shape
        new_img = np.zeros((3,H,W))
        for i in range(3):
            new_img[i,:,:] = img[:,:,i]
        return new_img

    def _get_dataframe(self):
        control_input_df = self._get_control_input_df()
        filename_df = self._get_image_path_df()
        df = control_input_df.merge(right=filename_df,
                                    left_on=['trajectory','input_num'],
                                    right_on=['trajectory', 'index'])
        df = df.drop_duplicates() # I have no idea why we have dupes
        
        stationary_mask = df['img1_t0'] == df['img1_t1']
        ctr1_col = df['ctr1'].copy()
        ctr2_col = df['ctr2'].copy()
        ctr1_col[stationary_mask] = 0
        ctr2_col[stationary_mask] = 0
        df['ctr1'] = ctr1_col
        df['ctr2'] = ctr2_col
        return df

    def _get_control_input_df(self):
        # xcxc I'm also assuming that our columns in control_input stay static like so.
        control_input_df = pd.read_csv(os.path.join(self.data_dir, 'control_input.txt'),
                               names=['trajectory', 'input_num', 'ctr1', 'ctr2'])
        control_input_df['input_num'] = control_input_df['input_num'].astype('int')
        control_input_df['trajectory'] = control_input_df['trajectory'].astype('str')
        return control_input_df
    
    def _get_image_path_df(self):
        filename_groupings = self._get_filename_groupings()
        filename_df = self._get_initial_filename_dataframe(filename_groupings)
        timestep_df = self._get_filename_dataframe_with_steps(filename_df)
        return timestep_df
    
    def _get_files_in_out(self):
        full_data = glob.glob(os.path.join(self.data_dir, '_out', '**.png'))
        abbrev_data = [x.split('/')[-1] for x in full_data]
        return abbrev_data
    
    def _get_filename_groupings(self):
        '''
        Reads in all the filenames, then groups them by
        (trajectory, timestep): [images]
        '''
        # A little cryptic, but it just gets the list of all filenames
        all_files_in_out = self._get_files_in_out()
        
        # We can then make a map with our data...
        filename_groupings = {}
        for fn in all_files_in_out:
            # Apologies for the hardcoding
            fn_number = str(int(fn.split('_')[0]))
            trajectory_number = str(int(fn.split('_')[2].split('.')[0]))
            if (fn_number, trajectory_number) not in filename_groupings:
                filename_groupings[(fn_number, trajectory_number)] = []
            filename_groupings[(fn_number, trajectory_number)].append(fn)
        return filename_groupings
    
    def _get_initial_filename_dataframe(self, filename_groupings):
        '''
        Given the filename groupings from the above, create a dataframe
        of the schema [trajectory, index, image1, image2]
        '''
        filename_df = pd.DataFrame(columns=['trajectory', 'index', 'img1', 'img2'])
        for k,v in filename_groupings.items():
            (index, traj) = k
            img1, img2 = None, None
            v.sort()
            if len(v) == 2:
                img1, img2 = v[0], v[1]
            elif len(v) == 1:
                img1 = v[0]
            filename_df = filename_df.append({
                'trajectory': traj,
                'index': index,
                'img1': img1,
                'img2': img2
            }, ignore_index=True)
        filename_df['trajectory'] = filename_df['trajectory'].astype('str')
        filename_df['index'] = filename_df['index'].astype('int')
        filename_df = filename_df.dropna(subset=['img1','img2']) # Drop if any of our images is None.
        return filename_df
    
    def _get_filename_dataframe_with_steps(self, filename_df):
        '''
        Given the filename df from the above, loop through it and get 
        the (t,t) and (t, t+1) pairings.
        '''
        schema = [
            'trajectory',
            'index',
            'img1_t0',
            'img2_t0',
            'img1_t1',
            'img2_t1']
        timestep_df = pd.DataFrame(columns=schema)
        num_rows, _ = filename_df.shape
        for i in range(num_rows): # blah blah yeah i know i'm not vectorizing. 
            ith_row = filename_df.iloc[i]
            row_dict = {
                'trajectory': ith_row['trajectory'],
                'index': ith_row['index'],
                'img1_t0': ith_row['img1'],
                'img2_t0': ith_row['img2']
            }
            # First, construct the stationary row
            stationary_row = row_dict.copy()
            stationary_row['img1_t1'] = ith_row['img1']
            stationary_row['img2_t1'] = ith_row['img2']
            timestep_df = timestep_df.append(stationary_row, ignore_index=True)
            # Then, construct the t+1th row IF it exists
            next_timestep_row = self._construct_next_timestep_row(
                row_dict, ith_row, filename_df)
            timestep_df = timestep_df.append(next_timestep_row, ignore_index=True)
        timestep_df['trajectory'] = timestep_df['trajectory'].astype('str')
        timestep_df['index'] = timestep_df['index'].astype('int')
        return timestep_df

    def _construct_next_timestep_row(self, row_dict, ith_row, filename_df):
        delta = 4
        next_index = ith_row['index'] + delta
        mask = (filename_df['index'] == next_index) & (filename_df['trajectory'] == ith_row['trajectory'])
        res = filename_df[mask]
        num_results = len(res)
        if num_results > 1:
            print("THERE'S MORE THAN ONE RESULT WHEN MAKING THE DATAFRAME")
        if num_results == 1:
            next_step_row = row_dict.copy()
            next_step_row['img1_t1'] = res['img1'].values[0]
            next_step_row['img2_t1'] = res['img2'].values[0]
            return next_step_row
        return None

### Model: `siameseCVAE.py` (xcxc To be changed later)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')     

In [0]:
class siameseCVAE(nn.Module):
	def __init__(self,batch=4):
		super().__init__()
		d = 0.4
		self.z_size = 64
		self.small = 256
		self.hidden = 1024
		ch_sz = 1
		c1 = 4
		c2 = 16
		last_conv = 16
		self.tensor = (batch,last_conv,75,100)
		flat = np.prod(self.tensor)
		flat2 = flat*2

		# channel_in, c_out, kernel_size, stride, padding
		def convbn(ci,co,ksz,s=1,pz=0):		#ReLu nonlinearity
			return nn.Sequential(
				nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
				nn.ReLU(),
				nn.BatchNorm2d(co))
		def convout(ci,co,ksz,s=1,pz=0):	#Sigmoid nonlinearity
			return nn.Sequential(
				nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
				nn.Sigmoid(),
				nn.BatchNorm2d(co))
		def mlp(in_size,hidden):
			return nn.Sequential(
				nn.Dropout(d),
				nn.Linear(in_size,hidden),
				nn.ReLU())

		#Encoder NN
		self.encx = nn.Sequential(
				nn.Dropout(d),
				convbn(ch_sz,c1,3,1,1),
				convbn(c1,c2,3,1,1),
				convbn(c2,last_conv,3,1,1))
		self.ency = nn.Sequential(
				nn.Dropout(d),
				convbn(ch_sz,c1,3,1,1),
				convbn(c1,c2,3,1,1),
				convbn(c2,last_conv,3,1,1))
		self.m1 = nn.Sequential(
				nn.Dropout(d),
				mlp(flat2,self.hidden),
				mlp(self.hidden, self.small))
		self.zmean = nn.Linear(self.small,self.z_size)
		self.zlogvar = nn.Linear(self.small,self.z_size)

		#Decoder NN
		self.expand_z = nn.Linear(self.z_size,self.small)
		self.mx = nn.Sequential(
				nn.Dropout(d),
				mlp(self.small,self.hidden),
				mlp(self.hidden,flat))
		self.my = nn.Sequential(
				nn.Dropout(d),
				mlp(self.small,self.hidden),
				mlp(self.hidden,flat))
		self.decx = nn.Sequential(
				nn.Dropout(d),
				convbn(last_conv,c2,3,1,1),
				convbn(c2,c1,3,1,1),
				convout(c1,ch_sz,3,1,1))
		self.decy = nn.Sequential(
				nn.Dropout(d),
				convbn(last_conv,c2,3,1,1),
				convbn(c2,c1,3,1,1),
				convout(c1,ch_sz,3,1,1))

	def encoder(self, x, y):
		# Flatten enc output
		h_x = self.encx(x).view(-1)
		h_y = self.ency(y).view(-1)
		# Concatenate flat convs
		h_layer = torch.cat((h_x,h_y))
		h = self.m1(h_layer)
		return h

	def bottleneck(self, x):
		z_mean = self.zmean(x)
		z_logvar = self.zlogvar(x)
		#reparam to get z latent sample
		std = torch.exp(0.5*z_logvar)
		eps = torch.randn_like(std)
		z = z_mean + eps*std
		return z, z_mean, z_logvar

	def decoder(self, z):
		#check the nonlinearities of this layer
		h = self.expand_z(z)
		#exand z to each decoder head
		h_x = self.mx(h)
		h_y = self.my(h)
		#make sure to reshape data correctly and decode
		x = self.decx(h_x.view(self.tensor))
		y = self.decy(h_x.view(self.tensor))
		return x, y

	def forward(self, x, y):
		h = self.encoder(x, y)
		z, z_mean, z_logvar = self.bottleneck(h)
		x_hat, y_hat = self.decoder(z)
		return x_hat, y_hat, z, z_mean, z_logvar

	def encode_get_z(self, x, y):
		h = self.encoder(x, y)
		z, z_mean, z_logvar = self.bottleneck(h)
		return z_mean

In [0]:
def ELBO_loss(xhat, x, yhat, y, mu, logvar):
    mseloss = nn.MSELoss(reduction='sum')
    MSE_X = mseloss(xhat, x)
    MSE_Y = mseloss(yhat, y)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE_X+MSE_Y+KLD, MSE_X, MSE_Y, KLD

In [0]:
transform = transforms.Compose([
        transforms.Resize((75,100)),
        transforms.ToTensor()])
batch = 1
path = "/content/drive/My Drive/Colab_Notebooks/ESE546_DL_Colab/project/double_camera"
dl = DataLoader(CarlaDataset(path, load_as_grayscale=True, transform=transform), batch_size=batch)

Evaluate and interpolate a Checkpoint

In [0]:
def interpolate(endpts,disc=10):
    zees = []
    Ls = []
    Rs = []
    for i in range(endpts.shape[1]):
        zees.append(np.linspace(endpts[0,i],endpts[1,i],num=disc))
    zees = np.asarray(zees)

    for j in range(disc):
        Lout, Rout = model.decoder(torch.from_numpy(zees[:,j]).float().to(device))
        L = Lout.cpu().detach().squeeze().numpy()
        R = Rout.cpu().detach().squeeze().numpy()
        Ls.append(L)
        Rs.append(R) 
    Ls = np.asarray(Ls)
    Rs = np.asarray(Rs)
    return Ls, Rs

def evaluate(net, disc):
    model = net.to(device)    
    left = []
    right = []
    recon_left = []
    recon_right = []
    z_endpts = []
    startend = [0,50]

    for i, X in enumerate(dl):
        model.eval()
        left_image_t = (X[0][:, 1, :, :, :]).float().to(device) # left/right images of t
        right_image_t = (X[0][:, 0, :, :, :]).float().to(device) # left/right images of t

        if i in startend:
            #Forward Pass
            xhat, yhat, z, z_mean, z_stdev = model.forward(left_image_t,right_image_t)

            # z_mean = model.encode_get_z(left_image_t,right_image_t)
            z_endpts.append(z.cpu().detach().numpy())

            left_ = left_image_t.cpu().squeeze().numpy()
            right_ = right_image_t.cpu().squeeze().numpy()
            xhat_ = xhat.cpu().detach().squeeze().numpy()
            yhat_ = yhat.cpu().detach().squeeze().numpy()

        left.append(left_)
        right.append(right_)
        recon_left.append(xhat_)
        recon_right.append(yhat_)
        if i == startend[1]:
            break
    left = np.asarray(left)
    right = np.asarray(right)
    recon_left = np.asarray(recon_left)
    recon_right = np.asarray(recon_right)
    z_endpts = np.asarray(z_endpts)

    # Interpolate between the endpoints and get intermediate images
    Ls, Rs = interpolate(z_endpts, disc=disc)

    for k in range(Ls.shape[0]):
        plt.figure()
        plt.subplot(121)
        plt.imshow(Ls[k,:,:], cmap = 'gray')
        plt.subplot(122)
        plt.imshow(Rs[k,:,:], cmap = 'gray')

    plt.figure()
    plt.subplot(121)
    plt.imshow(left[startend[0]], cmap = 'gray')
    plt.subplot(122)
    plt.imshow(right[startend[0]], cmap = 'gray')
    # plt.subplot(223)
    # plt.imshow(recon_left[startend[0]], cmap = 'gray')
    # plt.subplot(224)
    # plt.imshow(recon_right[startend[0]], cmap = 'gray')
    plt.figure()
    plt.subplot(121)
    plt.imshow(left[startend[1]], cmap = 'gray')
    plt.subplot(122)
    plt.imshow(right[startend[1]], cmap = 'gray')
    # plt.subplot(223)
    # plt.imshow(recon_left[startend[1]], cmap = 'gray')
    # plt.subplot(224)
    # plt.imshow(recon_right[startend[1]], cmap = 'gray')


#Run from here
model = siameseCVAE(batch=batch)
checkpoint = torch.load('/content/drive/My Drive/Colab_Notebooks/ESE546_DL_Colab/project/checkpoints/siamese_41616_smalllr.pt')
model.load_state_dict(checkpoint) 
disc = 25
evaluate(model,disc)